In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from tqdm import tqdm
import os
from datetime import datetime

In [2]:
if not os.path.isdir("data"):
    os.mkdir("data")

In [3]:
actual_draft_url = "https://www.nbadraft.net/actual-draft/?year-mock="

In [4]:
big_board_url = "https://www.nbadraft.net/ranking/bigboard/?year-ranking="

In [5]:
def height_to_inches(height: str) -> int:
    feet_inches = height.split("-")
    feet = int(feet_inches[0])
    inches = int(feet_inches[1])
    return feet * 12 + inches

In [6]:
def age_at_draft(birthday, draft_date):
    birthday = datetime.strptime(birthday, "%m/%d/%y")
    return round((draft_date - birthday).days / 365, 2)

# 2009

In [7]:
prospect_name_2009 = []
prospect_strengths_2009 = []
prospect_weaknesses_2009 = []
prospect_pick_2009 = []
prospect_height_2009 = []
prospect_weight_2009 = []
prospect_birthday_2009 = []
prospect_link_2009 = set()
actual_draft_2009 = BeautifulSoup(requests.get(actual_draft_url + "2009").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2009.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2009.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2009.add(link)
    prospect_name_2009.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2009.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2009.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2009.append(birthday)
    else:
        prospect_birthday_2009.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2009.append("")
        prospect_weaknesses_2009.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2009.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2009.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2009 = BeautifulSoup(requests.get(big_board_url + "2009").text, "html.parser")
for prospect_info in tqdm(big_board_2009.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2009:
        prospect_pick_2009.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2009.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2009.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2009.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2009.append(birthday)
        else:
            prospect_birthday_2009.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2009.append("")
            prospect_weaknesses_2009.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2009.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2009.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:47<00:00,  2.12it/s]


In [8]:
prospect_birthday_2009[23] = "02/14/89"
prospect_birthday_2009[26] = "07/27/86"
prospect_birthday_2009[47] = "04/18/86"
prospect_birthday_2009[73] = "03/31/85"
prospect_birthday_2009[101] = "04/11/88"

In [9]:
draft_2009_df =  pd.DataFrame({"Name": prospect_name_2009, "Pick": prospect_pick_2009,
                               "Strengths": prospect_strengths_2009, "Weaknesses": prospect_weaknesses_2009,
                               "Height": prospect_height_2009, "Weight": prospect_weight_2009, "Birthday": prospect_birthday_2009})
draft_2009_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Blake Griffin,1,"Freak of nature, off the charts athletic speci...",Still not a great shooter. Must become more co...,6-10,248,03/16/89
1,Hasheem Thabeet,2,Jim Calhoun called Thabeet “one of the most do...,Post game needs refining. Thabeet has not lear...,7-2,267,02/16/87
2,James Harden,3,Very smooth and crafty wing player … Used his ...,Ended the season with two very poor performanc...,6-5,222,08/26/89
3,Tyreke Evans,4,An extremely smooth and talented guard … He ha...,His jumpshot has shown very little progress an...,6-5,221,09/19/89
4,Ricky Rubio,5,One of the purest point guards to come around ...,Has good speed but lacks the great explosivene...,6-4,180,10/21/90
...,...,...,...,...,...,...,...
98,Lorrenzo Wade,-1,,,6-6,226,11/23/85
99,Eric Devendorf,-1,One of college basketballs top combo guards … ...,Lacks a true position at the next level. An un...,6-4,175,04/21/87
100,Lewis Clinch,-1,,,6-3,190,06/29/87
101,Dar Tucker,-1,,,6-3,200,04/11/88


In [10]:
draft_2009_df["Height"] = draft_2009_df["Height"].apply(height_to_inches)
draft_2009_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Blake Griffin,1,"Freak of nature, off the charts athletic speci...",Still not a great shooter. Must become more co...,82,248,03/16/89
1,Hasheem Thabeet,2,Jim Calhoun called Thabeet “one of the most do...,Post game needs refining. Thabeet has not lear...,86,267,02/16/87
2,James Harden,3,Very smooth and crafty wing player … Used his ...,Ended the season with two very poor performanc...,77,222,08/26/89
3,Tyreke Evans,4,An extremely smooth and talented guard … He ha...,His jumpshot has shown very little progress an...,77,221,09/19/89
4,Ricky Rubio,5,One of the purest point guards to come around ...,Has good speed but lacks the great explosivene...,76,180,10/21/90
...,...,...,...,...,...,...,...
98,Lorrenzo Wade,-1,,,78,226,11/23/85
99,Eric Devendorf,-1,One of college basketballs top combo guards … ...,Lacks a true position at the next level. An un...,76,175,04/21/87
100,Lewis Clinch,-1,,,75,190,06/29/87
101,Dar Tucker,-1,,,75,200,04/11/88


In [11]:
draft_2009_date = datetime(year = 2009, month = 6, day = 25)
draft_2009_df["Age"] = draft_2009_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2009_date))
draft_2009_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Blake Griffin,1,"Freak of nature, off the charts athletic speci...",Still not a great shooter. Must become more co...,82,248,03/16/89,20.29
1,Hasheem Thabeet,2,Jim Calhoun called Thabeet “one of the most do...,Post game needs refining. Thabeet has not lear...,86,267,02/16/87,22.37
2,James Harden,3,Very smooth and crafty wing player … Used his ...,Ended the season with two very poor performanc...,77,222,08/26/89,19.84
3,Tyreke Evans,4,An extremely smooth and talented guard … He ha...,His jumpshot has shown very little progress an...,77,221,09/19/89,19.78
4,Ricky Rubio,5,One of the purest point guards to come around ...,Has good speed but lacks the great explosivene...,76,180,10/21/90,18.69
...,...,...,...,...,...,...,...,...
98,Lorrenzo Wade,-1,,,78,226,11/23/85,23.60
99,Eric Devendorf,-1,One of college basketballs top combo guards … ...,Lacks a true position at the next level. An un...,76,175,04/21/87,22.19
100,Lewis Clinch,-1,,,75,190,06/29/87,22.01
101,Dar Tucker,-1,,,75,200,04/11/88,21.22


In [12]:
draft_2009_df.drop(columns = "Birthday").to_csv("data/2009_nba_draft_prospects.csv", index = None)

# 2010

In [13]:
prospect_name_2010 = []
prospect_strengths_2010 = []
prospect_weaknesses_2010 = []
prospect_pick_2010 = []
prospect_height_2010 = []
prospect_weight_2010 = []
prospect_birthday_2010 = []
prospect_link_2010 = set()
actual_draft_2010 = BeautifulSoup(requests.get(actual_draft_url + "2010").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2010.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2010.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2010.add(link)
    prospect_name_2010.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2010.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2010.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2010.append(birthday)
    else:
        prospect_birthday_2010.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2010.append("")
        prospect_weaknesses_2010.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2010.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2010.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2010 = BeautifulSoup(requests.get(big_board_url + "2010").text, "html.parser")
for prospect_info in tqdm(big_board_2010.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2010:
        prospect_pick_2010.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2010.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2010.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2010.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2010.append(birthday)
        else:
            prospect_birthday_2010.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2010.append("")
            prospect_weaknesses_2010.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2010.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2010.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:47<00:00,  2.08it/s]


In [14]:
prospect_birthday_2010[46] = "01/18/91"
prospect_birthday_2010[54] = "10/24/87"
prospect_birthday_2010[56] = "10/30/86"
prospect_birthday_2010[59] = "04/13/88"
prospect_birthday_2010[92] = "10/22/88"

In [15]:
draft_2010_df = pd.DataFrame({"Name": prospect_name_2010, "Pick": prospect_pick_2010,
                              "Strengths": prospect_strengths_2010, "Weaknesses": prospect_weaknesses_2010,
                              "Height": prospect_height_2010, "Weight": prospect_weight_2010, "Birthday": prospect_birthday_2010})

In [16]:
draft_2010_df["Height"] = draft_2010_df["Height"].apply(height_to_inches)
draft_2010_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,John Wall,1,One of the most electrifying players to come o...,"Similar to Derrick Rose entering the league, W...",76,196,09/06/90
1,Evan Turner,2,Big combo guard with great versatility and a t...,"Not a freak athlete, but above average by NBA ...",79,214,10/27/88
2,Derrick Favors,3,Explosive athlete with unique combination of p...,"Has done a good job adding bulk to his frame, ...",82,245,07/15/91
3,Wesley Johnson,4,"Wes has all the tools, and fits the descriptio...",Sometimes Wes settles or relies too much on hi...,79,206,07/11/87
4,DeMarcus Cousins,5,A physical mismatch in almost every sense. Cou...,Cousins’ lack of maturity and mental focus is ...,83,292,08/13/90
...,...,...,...,...,...,...,...
98,Dominique Archie,-1,,,79,200,08/19/87
99,Boban Marjanovic,-1,Bigman with a huge frame and solid athletic ab...,"A little bit stiff, does not change direction ...",87,293,08/15/88
100,Vladimir Dasic,-1,One of the most talented European prospects … ...,"For a player with his athleticism, Dasic relie...",82,226,05/13/88
101,Patrick Christopher,-1,Athletic wing player who has become a very rel...,"He is strictly a SF at this point, but at 6-5 ...",77,210,06/03/88


In [17]:
draft_2010_date = datetime(year = 2010, month = 6, day = 24)
draft_2010_df["Age"] = draft_2010_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2010_date))
draft_2010_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,John Wall,1,One of the most electrifying players to come o...,"Similar to Derrick Rose entering the league, W...",76,196,09/06/90,19.81
1,Evan Turner,2,Big combo guard with great versatility and a t...,"Not a freak athlete, but above average by NBA ...",79,214,10/27/88,21.67
2,Derrick Favors,3,Explosive athlete with unique combination of p...,"Has done a good job adding bulk to his frame, ...",82,245,07/15/91,18.96
3,Wesley Johnson,4,"Wes has all the tools, and fits the descriptio...",Sometimes Wes settles or relies too much on hi...,79,206,07/11/87,22.97
4,DeMarcus Cousins,5,A physical mismatch in almost every sense. Cou...,Cousins’ lack of maturity and mental focus is ...,83,292,08/13/90,19.88
...,...,...,...,...,...,...,...,...
98,Dominique Archie,-1,,,79,200,08/19/87,22.86
99,Boban Marjanovic,-1,Bigman with a huge frame and solid athletic ab...,"A little bit stiff, does not change direction ...",87,293,08/15/88,21.87
100,Vladimir Dasic,-1,One of the most talented European prospects … ...,"For a player with his athleticism, Dasic relie...",82,226,05/13/88,22.13
101,Patrick Christopher,-1,Athletic wing player who has become a very rel...,"He is strictly a SF at this point, but at 6-5 ...",77,210,06/03/88,22.07


In [18]:
draft_2010_df.drop(columns = "Birthday").to_csv("data/2010_nba_draft_prospects.csv", index = None)

# 2011

In [19]:
prospect_name_2011 = []
prospect_strengths_2011 = []
prospect_weaknesses_2011 = []
prospect_pick_2011 = []
prospect_height_2011 = []
prospect_weight_2011 = []
prospect_birthday_2011 = []
prospect_link_2011 = set()
actual_draft_2011 = BeautifulSoup(requests.get(actual_draft_url + "2011").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2011.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    if pick >= 9:
        pick += 1
    prospect_pick_2011.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2011.add(link)
    prospect_name_2011.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2011.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2011.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2011.append(birthday)
    else:
        prospect_birthday_2011.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2011.append("")
        prospect_weaknesses_2011.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2011.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2011.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2011 = BeautifulSoup(requests.get(big_board_url + "2011").text, "html.parser")
for prospect_info in tqdm(big_board_2011.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2011:
        if link == "https://www.nbadraft.net/players/jimmer-fredette/":
            prospect_pick_2011.append(10)
        else:
            prospect_pick_2011.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2011.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2011.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2011.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2011.append(birthday)
        else:
            prospect_birthday_2011.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2011.append("")
            prospect_weaknesses_2011.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2011.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2011.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:54<00:00,  1.82it/s]


In [20]:
prospect_birthday_2011[34] = "10/11/90"
prospect_birthday_2011[48] = "02/04/89"
prospect_birthday_2011[85] = "04/23/88"
prospect_birthday_2011[102] = "01/08/88"

In [21]:
draft_2011_df = pd.DataFrame({"Name": prospect_name_2011, "Pick": prospect_pick_2011,
                              "Strengths": prospect_strengths_2011, "Weaknesses": prospect_weaknesses_2011,
                              "Height": prospect_height_2011, "Weight": prospect_weight_2011, "Birthday": prospect_birthday_2011})

In [22]:
draft_2011_df["Height"] = draft_2011_df["Height"].apply(height_to_inches)
draft_2011_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Kyrie Irving,1,A “true” PG with a great feel for the game … C...,Durability is a legitimate concern after he mi...,75,191,03/23/92
1,Derrick Williams,2,Williams’ production in his 2 years at Arizona...,"Considered a combo forward, and there are some...",80,248,05/25/91
2,Enes Kanter,3,"Bigman with excellent size, strength and polis...",One of the big concerns scouts have with Kante...,83,259,05/20/92
3,Tristan Thompson,4,A lefty post player with a tremendous wingspan...,At 6-9 he is a bit on the short side as a powe...,81,227,03/13/91
4,Jonas Valanciunas,5,Lithuanian bigman with great size and reboundi...,Can’t create his own offense yet … Has a buddi...,83,240,05/06/92
...,...,...,...,...,...,...,...
102,Alex Tyus,-1,Athletic and energetic defensive-minded forwar...,No real offensive game… His scoring repertoire...,80,207,01/08/88
103,Jeremy Hazell,-1,,,77,185,03/25/86
104,Clarence Jackson,-1,,,76,188,04/05/89
105,Ferrakohn Hall,-1,,,80,225,09/20/90


In [23]:
draft_2011_date = datetime(year = 2011, month = 6, day = 23)
draft_2011_df["Age"] = draft_2011_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2011_date))
draft_2011_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Kyrie Irving,1,A “true” PG with a great feel for the game … C...,Durability is a legitimate concern after he mi...,75,191,03/23/92,19.26
1,Derrick Williams,2,Williams’ production in his 2 years at Arizona...,"Considered a combo forward, and there are some...",80,248,05/25/91,20.09
2,Enes Kanter,3,"Bigman with excellent size, strength and polis...",One of the big concerns scouts have with Kante...,83,259,05/20/92,19.10
3,Tristan Thompson,4,A lefty post player with a tremendous wingspan...,At 6-9 he is a bit on the short side as a powe...,81,227,03/13/91,20.29
4,Jonas Valanciunas,5,Lithuanian bigman with great size and reboundi...,Can’t create his own offense yet … Has a buddi...,83,240,05/06/92,19.14
...,...,...,...,...,...,...,...,...
102,Alex Tyus,-1,Athletic and energetic defensive-minded forwar...,No real offensive game… His scoring repertoire...,80,207,01/08/88,23.47
103,Jeremy Hazell,-1,,,77,185,03/25/86,25.26
104,Clarence Jackson,-1,,,76,188,04/05/89,22.23
105,Ferrakohn Hall,-1,,,80,225,09/20/90,20.77


In [24]:
draft_2011_df.drop(columns = "Birthday").to_csv("data/2011_nba_draft_prospects.csv", index = None)

# 2012

In [25]:
prospect_name_2012 = []
prospect_strengths_2012 = []
prospect_weaknesses_2012 = []
prospect_pick_2012 = []
prospect_height_2012 = []
prospect_weight_2012 = []
prospect_birthday_2012 = []
prospect_link_2012 = set()
actual_draft_2012 = BeautifulSoup(requests.get(actual_draft_url + "2012").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2012.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2012.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2012.add(link)
    prospect_name_2012.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2012.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2012.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2012.append(birthday)
    else:
        prospect_birthday_2012.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1 or pick + 1 == 50:
        prospect_strengths_2012.append("")
        prospect_weaknesses_2012.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2012.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2012.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2012 = BeautifulSoup(requests.get(big_board_url + "2012").text, "html.parser")
for prospect_info in tqdm(big_board_2012.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2012:
        prospect_pick_2012.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2012.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2012.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2012.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2012.append(birthday)
        else:
            prospect_birthday_2012.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2012.append("")
            prospect_weaknesses_2012.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2012.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2012.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 99/99 [01:07<00:00,  1.46it/s]


In [26]:
prospect_birthday_2012[51] = "05/16/90"
prospect_birthday_2012[68] = "03/27/90"
prospect_birthday_2012[72] = "05/27/89"
prospect_birthday_2012[73] = "06/08/90"
prospect_birthday_2012[93] = "01/17/90"
prospect_birthday_2012[98] = "11/06/89"

In [27]:
draft_2012_df = pd.DataFrame({"Name": prospect_name_2012, "Pick": prospect_pick_2012,
                              "Strengths": prospect_strengths_2012, "Weaknesses": prospect_weaknesses_2012,
                              "Height": prospect_height_2012, "Weight": prospect_weight_2012, "Birthday": prospect_birthday_2012})

In [28]:
draft_2012_df["Height"] = draft_2012_df["Height"].apply(height_to_inches)
draft_2012_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Anthony Davis,1,"Fluid athleticism, incredible length and guard...",The number one drawback on Davis is his overal...,82,222,03/11/93
1,Michael Kidd-Gilchrist,2,"MKG is a strong, athletic small forward who po...","Jack of all trades, master of none … MKG’s big...",79,233,09/26/93
2,Bradley Beal,3,Talented shooting guard with a prototypical sk...,May be an inch or two shorter than most starti...,76,202,06/28/93
3,Dion Waiters,4,Waiters’ style of play reflects your prototypi...,"He’s clearly not a point guard, so you wonder ...",76,215,12/10/91
4,Thomas Robinson,5,High-motor forward that possesses the power/at...,Doesn’t excel at any one area offensively … Hi...,81,244,03/18/91
...,...,...,...,...,...,...,...
97,Kent Bazemore,-1,,,77,201,07/01/89
98,Larry Anderson,-1,,,77,211,11/06/89
99,Dario Hunt,-1,,,80,230,05/02/89
100,Kyle Fogg,-1,,,75,183,01/27/90


In [29]:
draft_2012_date = datetime(year = 2012, month = 6, day = 28)
draft_2012_df["Age"] = draft_2012_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2012_date))
draft_2012_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Anthony Davis,1,"Fluid athleticism, incredible length and guard...",The number one drawback on Davis is his overal...,82,222,03/11/93,19.31
1,Michael Kidd-Gilchrist,2,"MKG is a strong, athletic small forward who po...","Jack of all trades, master of none … MKG’s big...",79,233,09/26/93,18.77
2,Bradley Beal,3,Talented shooting guard with a prototypical sk...,May be an inch or two shorter than most starti...,76,202,06/28/93,19.01
3,Dion Waiters,4,Waiters’ style of play reflects your prototypi...,"He’s clearly not a point guard, so you wonder ...",76,215,12/10/91,20.56
4,Thomas Robinson,5,High-motor forward that possesses the power/at...,Doesn’t excel at any one area offensively … Hi...,81,244,03/18/91,21.30
...,...,...,...,...,...,...,...,...
97,Kent Bazemore,-1,,,77,201,07/01/89,23.01
98,Larry Anderson,-1,,,77,211,11/06/89,22.66
99,Dario Hunt,-1,,,80,230,05/02/89,23.17
100,Kyle Fogg,-1,,,75,183,01/27/90,22.43


In [30]:
draft_2012_df.drop(columns = "Birthday").to_csv("data/2012_nba_draft_prospects.csv", index = None)

# 2013

In [31]:
prospect_name_2013 = []
prospect_strengths_2013 = []
prospect_weaknesses_2013 = []
prospect_pick_2013 = []
prospect_height_2013 = []
prospect_weight_2013 = []
prospect_birthday_2013 = []
prospect_link_2013 = set()
actual_draft_2013 = BeautifulSoup(requests.get(actual_draft_url + "2013").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2013.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2013.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2013.add(link)
    prospect_name_2013.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2013.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2013.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2013.append(birthday)
    else:
        prospect_birthday_2013.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2013.append("")
        prospect_weaknesses_2013.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2013.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2013.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2013 = BeautifulSoup(requests.get(big_board_url + "2013").text, "html.parser")
for prospect_info in tqdm(big_board_2013.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2013:
        prospect_pick_2013.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2013.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2013.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2013.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2013.append(birthday)
        else:
            prospect_birthday_2013.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2013.append("")
            prospect_weaknesses_2013.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2013.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2013.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:44<00:00,  2.25it/s]


In [32]:
draft_2013_df = pd.DataFrame({"Name": prospect_name_2013, "Pick": prospect_pick_2013,
                              "Strengths": prospect_strengths_2013, "Weaknesses": prospect_weaknesses_2013,
                              "Height": prospect_height_2013, "Weight": prospect_weight_2013, "Birthday": prospect_birthday_2013})

In [33]:
draft_2013_df["Height"] = draft_2013_df["Height"].apply(height_to_inches)
draft_2013_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Anthony Bennett,1,Bennett has expanded his game offensively and ...,Somewhat of a tweener as he is slightly unders...,79,240,03/14/93
1,Victor Oladipo,2,Explosive athlete who plays with great energy ...,Still very much a work in progress as a comple...,76,213,05/04/92
2,Otto Porter,3,Porter is a multi-skilled small forward posses...,Smooth run/jump athlete but missing the tradem...,81,200,06/03/93
3,Cody Zeller,4,Bigman with an excellent basketball IQ and all...,Not a shot blocker or much of a presence on th...,84,230,10/05/92
4,Alex Len,5,Athletic specimen at 7’1 with highly intriguin...,Len remains a raw product when forced to make ...,85,255,06/16/93
...,...,...,...,...,...,...,...
96,Elijah Johnson,-1,A combo guard with excellent physical tools. H...,All of Johnson’s weaknesses seem to stem from ...,74,195,07/11/90
97,Kenny Boynton,-1,Has the ability to score in bunches and take o...,Was a top 10 recruit throughout his high schoo...,74,190,05/12/91
98,Devin Booker Clemson,-1,,,80,250,02/28/91
99,Ed Daniel,-1,,,79,225,06/13/90


In [34]:
draft_2013_date = datetime(year = 2013, month = 6, day = 27)
draft_2013_df["Age"] = draft_2013_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2013_date))
draft_2013_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Anthony Bennett,1,Bennett has expanded his game offensively and ...,Somewhat of a tweener as he is slightly unders...,79,240,03/14/93,20.30
1,Victor Oladipo,2,Explosive athlete who plays with great energy ...,Still very much a work in progress as a comple...,76,213,05/04/92,21.16
2,Otto Porter,3,Porter is a multi-skilled small forward posses...,Smooth run/jump athlete but missing the tradem...,81,200,06/03/93,20.08
3,Cody Zeller,4,Bigman with an excellent basketball IQ and all...,Not a shot blocker or much of a presence on th...,84,230,10/05/92,20.74
4,Alex Len,5,Athletic specimen at 7’1 with highly intriguin...,Len remains a raw product when forced to make ...,85,255,06/16/93,20.04
...,...,...,...,...,...,...,...,...
96,Elijah Johnson,-1,A combo guard with excellent physical tools. H...,All of Johnson’s weaknesses seem to stem from ...,74,195,07/11/90,22.98
97,Kenny Boynton,-1,Has the ability to score in bunches and take o...,Was a top 10 recruit throughout his high schoo...,74,190,05/12/91,22.14
98,Devin Booker Clemson,-1,,,80,250,02/28/91,22.34
99,Ed Daniel,-1,,,79,225,06/13/90,23.05


In [35]:
draft_2013_df.drop(columns = "Birthday").to_csv("data/2013_nba_draft_prospects.csv", index = None)

# 2014

In [36]:
prospect_name_2014 = []
prospect_strengths_2014 = []
prospect_weaknesses_2014 = []
prospect_pick_2014 = []
prospect_height_2014 = []
prospect_weight_2014 = []
prospect_birthday_2014 = []
prospect_link_2014 = set()
actual_draft_2014 = BeautifulSoup(requests.get(actual_draft_url + "2014").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2014.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2014.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2014.add(link)
    prospect_name_2014.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2014.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2014.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2014.append(birthday)
    else:
        prospect_birthday_2014.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2014.append("")
        prospect_weaknesses_2014.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2014.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2014.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2014 = BeautifulSoup(requests.get(big_board_url + "2014").text, "html.parser")
for prospect_info in tqdm(big_board_2014.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2014:
        prospect_pick_2014.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2014.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2014.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2014.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2014.append(birthday)
        else:
            prospect_birthday_2014.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2014.append("")
            prospect_weaknesses_2014.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2014.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2014.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:43<00:00,  2.28it/s]


In [37]:
draft_2014_df = pd.DataFrame({"Name": prospect_name_2014, "Pick": prospect_pick_2014,
                              "Strengths": prospect_strengths_2014, "Weaknesses": prospect_weaknesses_2014,
                              "Height": prospect_height_2014, "Weight": prospect_weight_2014, "Birthday": prospect_birthday_2014})

In [38]:
draft_2014_df["Height"] = draft_2014_df["Height"].apply(height_to_inches)
draft_2014_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Andrew Wiggins,1,Off the charts run/jump athleticism … Lateral ...,The biggest issue is whether or not Wiggins ha...,80,200,02/23/95
1,Jabari Parker,2,"Parker is on another level from his peers, ski...",Defense and position are the two biggest quest...,81,240,03/15/95
2,Joel Embiid,3,The sculpted Embiid has a prototypical body fo...,"Embiid had trouble staying on the floor, eithe...",84,250,03/16/94
3,Aaron Gordon,4,A very athletic and versatile PF … Is a very a...,Gordon’s main weaknesses revolve around his la...,81,220,09/16/95
4,Dante Exum,5,The 18-year-old Aussie is a fluid and versatil...,Level of competition is the biggest knock agai...,78,196,07/13/95
...,...,...,...,...,...,...,...
99,Roberto Nelson,-1,,,75,195,03/06/91
100,Aaron Craft,-1,Excellent on-ball defender with an incredible ...,Doesn’t have NBA measurable or athleticism. H...,74,192,02/12/92
101,Bryce Cotton,-1,,,72,165,08/11/92
102,Alex Kirk,-1,"A versatile offensive threat, Kirk averaged a ...",Not overly physical on either end of the floor...,82,252,11/14/91


In [39]:
draft_2014_date = datetime(year = 2014, month = 6, day = 26)
draft_2014_df["Age"] = draft_2014_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2014_date))
draft_2014_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Andrew Wiggins,1,Off the charts run/jump athleticism … Lateral ...,The biggest issue is whether or not Wiggins ha...,80,200,02/23/95,19.35
1,Jabari Parker,2,"Parker is on another level from his peers, ski...",Defense and position are the two biggest quest...,81,240,03/15/95,19.30
2,Joel Embiid,3,The sculpted Embiid has a prototypical body fo...,"Embiid had trouble staying on the floor, eithe...",84,250,03/16/94,20.29
3,Aaron Gordon,4,A very athletic and versatile PF … Is a very a...,Gordon’s main weaknesses revolve around his la...,81,220,09/16/95,18.79
4,Dante Exum,5,The 18-year-old Aussie is a fluid and versatil...,Level of competition is the biggest knock agai...,78,196,07/13/95,18.97
...,...,...,...,...,...,...,...,...
99,Roberto Nelson,-1,,,75,195,03/06/91,23.32
100,Aaron Craft,-1,Excellent on-ball defender with an incredible ...,Doesn’t have NBA measurable or athleticism. H...,74,192,02/12/92,22.38
101,Bryce Cotton,-1,,,72,165,08/11/92,21.89
102,Alex Kirk,-1,"A versatile offensive threat, Kirk averaged a ...",Not overly physical on either end of the floor...,82,252,11/14/91,22.63


In [40]:
draft_2014_df.drop(columns = "Birthday").to_csv("data/2014_nba_draft_prospects.csv", index = None)

# 2015

In [41]:
prospect_name_2015 = []
prospect_strengths_2015 = []
prospect_weaknesses_2015 = []
prospect_pick_2015 = []
prospect_height_2015 = []
prospect_weight_2015 = []
prospect_birthday_2015 = []
prospect_link_2015 = set()
actual_draft_2015 = BeautifulSoup(requests.get(actual_draft_url + "2015").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2015.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2015.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2015.add(link)
    prospect_name_2015.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2015.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2015.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2015.append(birthday)
    else:
        prospect_birthday_2015.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2015.append("")
        prospect_weaknesses_2015.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2015.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2015.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2015 = BeautifulSoup(requests.get(big_board_url + "2015").text, "html.parser")
for prospect_info in tqdm(big_board_2015.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2015:
        prospect_pick_2015.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2015.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2015.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2015.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2015.append(birthday)
        else:
            prospect_birthday_2015.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2015.append("")
            prospect_weaknesses_2015.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2015.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2015.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:45<00:00,  2.19it/s]


In [42]:
prospect_birthday_2015[98] = "09/14/93"
prospect_birthday_2015[101] = "09/17/92"

In [43]:
draft_2015_df = pd.DataFrame({"Name": prospect_name_2015, "Pick": prospect_pick_2015,
                              "Strengths": prospect_strengths_2015, "Weaknesses": prospect_weaknesses_2015,
                              "Height": prospect_height_2015, "Weight": prospect_weight_2015, "Birthday": prospect_birthday_2015})

In [44]:
draft_2015_df["Height"] = draft_2015_df["Height"].apply(height_to_inches)
draft_2015_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Karl-Anthony Towns,1,Center prospect with the size and skill set to...,Has struggled mightily with his post offense e...,84,250,11/15/95
1,D'Angelo Russell,2,Precocious point guard with tremendous ball sk...,"Still unproven as point guard, so the question...",76,195,02/23/96
2,Jahlil Okafor,3,One of the most advanced low-post scorers the ...,While his conditioning at Duke was a great sig...,83,270,12/15/95
3,Kristaps Porzingis,4,It’s not easy to find a legit 7-footer with gr...,"The main issue for Pozingis is his strength, a...",85,230,08/02/95
4,Mario Hezonja,5,"Standing 6-8, he has ideal size for the wing p...",He still could work on his body strength to be...,80,215,02/25/95
...,...,...,...,...,...,...,...
97,Askia Booker,-1,,,74,170,08/31/93
98,Derrick Marks,-1,,,75,210,09/14/93
99,David Kravish,-1,,,82,240,09/12/92
100,LaDontae Henton,-1,"At 6-foot-6, 215, Henton is a tough and physic...",Heavy feet for a 3 … Does not possess great qu...,77,215,01/06/92


In [45]:
draft_2015_date = datetime(year = 2015, month = 6, day = 25)
draft_2015_df["Age"] = draft_2015_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2015_date))
draft_2015_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Karl-Anthony Towns,1,Center prospect with the size and skill set to...,Has struggled mightily with his post offense e...,84,250,11/15/95,19.62
1,D'Angelo Russell,2,Precocious point guard with tremendous ball sk...,"Still unproven as point guard, so the question...",76,195,02/23/96,19.35
2,Jahlil Okafor,3,One of the most advanced low-post scorers the ...,While his conditioning at Duke was a great sig...,83,270,12/15/95,19.54
3,Kristaps Porzingis,4,It’s not easy to find a legit 7-footer with gr...,"The main issue for Pozingis is his strength, a...",85,230,08/02/95,19.91
4,Mario Hezonja,5,"Standing 6-8, he has ideal size for the wing p...",He still could work on his body strength to be...,80,215,02/25/95,20.34
...,...,...,...,...,...,...,...,...
97,Askia Booker,-1,,,74,170,08/31/93,21.83
98,Derrick Marks,-1,,,75,210,09/14/93,21.79
99,David Kravish,-1,,,82,240,09/12/92,22.80
100,LaDontae Henton,-1,"At 6-foot-6, 215, Henton is a tough and physic...",Heavy feet for a 3 … Does not possess great qu...,77,215,01/06/92,23.48


In [46]:
draft_2015_df.drop(columns = "Birthday").to_csv("data/2015_nba_draft_prospects.csv", index = None)

# 2016

In [47]:
prospect_name_2016 = []
prospect_strengths_2016 = []
prospect_weaknesses_2016 = []
prospect_pick_2016 = []
prospect_height_2016 = []
prospect_weight_2016 = []
prospect_birthday_2016 = []
prospect_link_2016 = set()
actual_draft_2016 = BeautifulSoup(requests.get(actual_draft_url + "2016").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2016.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2016.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2016.add(link)
    prospect_name_2016.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2016.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2016.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2016.append(birthday)
    else:
        prospect_birthday_2016.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2016.append("")
        prospect_weaknesses_2016.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2016.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2016.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2016 = BeautifulSoup(requests.get(big_board_url + "2016").text, "html.parser")
for prospect_info in tqdm(big_board_2016.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2016:
        prospect_pick_2016.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2016.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2016.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2016.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2016.append(birthday)
        else:
            prospect_birthday_2016.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2016.append("")
            prospect_weaknesses_2016.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2016.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2016.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:48<00:00,  2.08it/s]


In [48]:
prospect_birthday_2016[53] = "03/29/95"
prospect_birthday_2016[91] = "10/17/92"
prospect_birthday_2016[101] = "11/03/93"

In [49]:
draft_2016_df = pd.DataFrame({"Name": prospect_name_2016, "Pick": prospect_pick_2016,
                              "Strengths": prospect_strengths_2016, "Weaknesses": prospect_weaknesses_2016,
                              "Height": prospect_height_2016, "Weight": prospect_weight_2016, "Birthday": prospect_birthday_2016})

In [50]:
draft_2016_df["Height"] = draft_2016_df["Height"].apply(height_to_inches)
draft_2016_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Ben Simmons,1,"An extremely unique talent, combining power fo...","Despite being a good scorer, he lacks a reliab...",82,240,07/20/96
1,Brandon Ingram,2,"Wingspan, wingspan, wingspan, Ingram is all ar...","Extremely frail, weighed 196 lbs at the 2015 N...",81,195,09/02/97
2,Jaylen Brown,3,Wing with great physical and athletic tools wi...,"Shot selection was definitely an issue, would ...",78,225,10/24/96
3,Dragan Bender,4,"Versatile player, who can play both forward po...",Not a great athlete … Limited leaping ability ...,84,220,11/17/97
4,Kris Dunn,5,Fantastic size for a PG with quickness and ath...,"Has been known to play out of control, was qui...",76,220,03/18/94
...,...,...,...,...,...,...,...
97,Stefan Jankovic,-1,,,83,235,08/04/93
98,Brannen Greene,-1,,,79,215,12/21/94
99,D'Vauntes Smith-Rivera,-1,,,75,215,12/20/92
100,Goodluck Okonoboh,-1,,,82,225,09/30/94


In [51]:
draft_2016_date = datetime(year = 2016, month = 6, day = 23)
draft_2016_df["Age"] = draft_2016_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2016_date))
draft_2016_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Ben Simmons,1,"An extremely unique talent, combining power fo...","Despite being a good scorer, he lacks a reliab...",82,240,07/20/96,19.94
1,Brandon Ingram,2,"Wingspan, wingspan, wingspan, Ingram is all ar...","Extremely frail, weighed 196 lbs at the 2015 N...",81,195,09/02/97,18.82
2,Jaylen Brown,3,Wing with great physical and athletic tools wi...,"Shot selection was definitely an issue, would ...",78,225,10/24/96,19.68
3,Dragan Bender,4,"Versatile player, who can play both forward po...",Not a great athlete … Limited leaping ability ...,84,220,11/17/97,18.61
4,Kris Dunn,5,Fantastic size for a PG with quickness and ath...,"Has been known to play out of control, was qui...",76,220,03/18/94,22.28
...,...,...,...,...,...,...,...,...
97,Stefan Jankovic,-1,,,83,235,08/04/93,22.90
98,Brannen Greene,-1,,,79,215,12/21/94,21.52
99,D'Vauntes Smith-Rivera,-1,,,75,215,12/20/92,23.52
100,Goodluck Okonoboh,-1,,,82,225,09/30/94,21.75


In [52]:
draft_2016_df.drop(columns = "Birthday").to_csv("data/2016_nba_draft_prospects.csv", index = None)

# 2017

In [53]:
prospect_name_2017 = []
prospect_strengths_2017 = []
prospect_weaknesses_2017 = []
prospect_pick_2017 = []
prospect_height_2017 = []
prospect_weight_2017 = []
prospect_birthday_2017 = []
prospect_link_2017 = set()
actual_draft_2017 = BeautifulSoup(requests.get(actual_draft_url + "2017").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2017.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2017.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2017.add(link)
    prospect_name_2017.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2017.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2017.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2017.append(birthday)
    else:
        prospect_birthday_2017.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2017.append("")
        prospect_weaknesses_2017.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2017.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2017.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2017 = BeautifulSoup(requests.get(big_board_url + "2017").text, "html.parser")
for prospect_info in tqdm(big_board_2017.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2017:
        prospect_pick_2017.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2017.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2017.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2017.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2017.append(birthday)
        else:
            prospect_birthday_2017.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2017.append("")
            prospect_weaknesses_2017.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2017.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2017.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:45<00:00,  2.20it/s]


In [54]:
draft_2017_df = pd.DataFrame({"Name": prospect_name_2017, "Pick": prospect_pick_2017,
                              "Strengths": prospect_strengths_2017, "Weaknesses": prospect_weaknesses_2017,
                              "Height": prospect_height_2017, "Weight": prospect_weight_2017, "Birthday": prospect_birthday_2017})

In [55]:
draft_2017_df["Height"] = draft_2017_df["Height"].apply(height_to_inches)
draft_2017_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Markelle Fultz,1,Fultz has elite physical tools for an NBA poin...,Fultz is a very hard worker off the court who ...,76,190,05/29/98
1,Lonzo Ball,2,"At 6’6"" with a 6’9"" wingspan, Ball has a uniqu...","Ball has an outstanding combination of size, l...",78,190,10/27/97
2,Jayson Tatum,3,Tatum lived up to lofty expectations placed up...,One of the first flaws that jump out in Tatum’...,80,205,03/03/98
3,Josh Jackson,4,One of the main things that jumps out at you i...,Jackson’s biggest weakness right now is his sh...,80,205,02/10/97
4,De'Aaron Fox,5,De’Aaron Fox is a kid that plays with a great ...,The biggest obstacle for Fox in his transition...,75,170,12/20/97
...,...,...,...,...,...,...,...
95,Xavier Rathan-Mayes,-1,,,75,210,04/29/94
96,Austin Nichols,-1,,,81,235,09/20/94
97,Ismael Bako,-1,Bako is a very long big man with some upside l...,"At 210 lbs, lacks strength and a physical pres...",83,210,10/10/95
98,Tyler Roberson,-1,,,80,225,11/27/94


In [56]:
draft_2017_date = datetime(year = 2017, month = 6, day = 22)
draft_2017_df["Age"] = draft_2017_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2017_date))
draft_2017_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Markelle Fultz,1,Fultz has elite physical tools for an NBA poin...,Fultz is a very hard worker off the court who ...,76,190,05/29/98,19.08
1,Lonzo Ball,2,"At 6’6"" with a 6’9"" wingspan, Ball has a uniqu...","Ball has an outstanding combination of size, l...",78,190,10/27/97,19.67
2,Jayson Tatum,3,Tatum lived up to lofty expectations placed up...,One of the first flaws that jump out in Tatum’...,80,205,03/03/98,19.32
3,Josh Jackson,4,One of the main things that jumps out at you i...,Jackson’s biggest weakness right now is his sh...,80,205,02/10/97,20.38
4,De'Aaron Fox,5,De’Aaron Fox is a kid that plays with a great ...,The biggest obstacle for Fox in his transition...,75,170,12/20/97,19.52
...,...,...,...,...,...,...,...,...
95,Xavier Rathan-Mayes,-1,,,75,210,04/29/94,23.16
96,Austin Nichols,-1,,,81,235,09/20/94,22.77
97,Ismael Bako,-1,Bako is a very long big man with some upside l...,"At 210 lbs, lacks strength and a physical pres...",83,210,10/10/95,21.72
98,Tyler Roberson,-1,,,80,225,11/27/94,22.58


In [57]:
draft_2017_df.drop(columns = "Birthday").to_csv("data/2017_nba_draft_prospects.csv", index = None)

# 2018

In [58]:
prospect_name_2018 = []
prospect_strengths_2018 = []
prospect_weaknesses_2018 = []
prospect_pick_2018 = []
prospect_height_2018 = []
prospect_weight_2018 = []
prospect_birthday_2018 = []
prospect_link_2018 = set()
actual_draft_2018 = BeautifulSoup(requests.get(actual_draft_url + "2018").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2018.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2018.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2018.add(link)
    prospect_name_2018.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2018.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2018.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2018.append(birthday)
    else:
        prospect_birthday_2018.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2018.append("")
        prospect_weaknesses_2018.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2018.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2018.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2018 = BeautifulSoup(requests.get(big_board_url + "2018").text, "html.parser")
for prospect_info in tqdm(big_board_2018.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2018:
        prospect_pick_2018.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2018.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2018.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2018.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2018.append(birthday)
        else:
            prospect_birthday_2018.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2018.append("")
            prospect_weaknesses_2018.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2018.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2018.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:48<00:00,  2.08it/s]


In [59]:
draft_2018_df = pd.DataFrame({"Name": prospect_name_2018, "Pick": prospect_pick_2018,
                              "Strengths": prospect_strengths_2018, "Weaknesses": prospect_weaknesses_2018,
                              "Height": prospect_height_2018, "Weight": prospect_weight_2018, "Birthday": prospect_birthday_2018})

In [60]:
draft_2018_df["Height"] = draft_2018_df["Height"].apply(height_to_inches)
draft_2018_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Deandre Ayton,1,"Tremendous physical profile (7’1”, 250 pounds)...",Defensive awareness needs to improve. For a pl...,84,260,07/23/98
1,Marvin Bagley,2,NBA-ready body … Positional versatility … Disp...,Lacks the wingspan of other big men in his cla...,83,235,03/14/99
2,Luka Doncic,3,Versatile all around guard/forward who can do ...,He can be considered magrinally average athlet...,80,225,02/28/99
3,Jaren Jackson Jr.,4,Very gifted athletically … Untapped potential ...,A late bloomer in regards to his physical fram...,83,235,09/15/99
4,Trae Young,5,Trae Young early on in his freshman season is ...,Young does not have ideal size at 6-2 or lengt...,74,180,09/19/98
...,...,...,...,...,...,...,...
96,Kyle Washington,-1,,,81,240,08/18/93
97,Chris Chiozza,-1,,,72,175,11/21/95
98,Anas Mahmoud,-1,,,84,215,05/09/95
99,Jock Landale,-1,,,83,255,10/25/95


In [61]:
draft_2018_date = datetime(year = 2018, month = 6, day = 21)
draft_2018_df["Age"] = draft_2018_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2018_date))
draft_2018_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Deandre Ayton,1,"Tremendous physical profile (7’1”, 250 pounds)...",Defensive awareness needs to improve. For a pl...,84,260,07/23/98,19.93
1,Marvin Bagley,2,NBA-ready body … Positional versatility … Disp...,Lacks the wingspan of other big men in his cla...,83,235,03/14/99,19.28
2,Luka Doncic,3,Versatile all around guard/forward who can do ...,He can be considered magrinally average athlet...,80,225,02/28/99,19.32
3,Jaren Jackson Jr.,4,Very gifted athletically … Untapped potential ...,A late bloomer in regards to his physical fram...,83,235,09/15/99,18.78
4,Trae Young,5,Trae Young early on in his freshman season is ...,Young does not have ideal size at 6-2 or lengt...,74,180,09/19/98,19.77
...,...,...,...,...,...,...,...,...
96,Kyle Washington,-1,,,81,240,08/18/93,24.86
97,Chris Chiozza,-1,,,72,175,11/21/95,22.60
98,Anas Mahmoud,-1,,,84,215,05/09/95,23.13
99,Jock Landale,-1,,,83,255,10/25/95,22.67


In [62]:
draft_2018_df.drop(columns = "Birthday").to_csv("data/2018_nba_draft_prospects.csv", index = None)

# 2019

In [63]:
prospect_name_2019 = []
prospect_strengths_2019 = []
prospect_weaknesses_2019 = []
prospect_pick_2019 = []
prospect_height_2019 = []
prospect_weight_2019 = []
prospect_birthday_2019 = []
prospect_link_2019 = set()
actual_draft_2019 = BeautifulSoup(requests.get(actual_draft_url + "2019").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2019.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2019.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2019.add(link)
    prospect_name_2019.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2019.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2019.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2019.append(birthday)
    else:
        prospect_birthday_2019.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if pick + 1 == 1:
        strengths_weaknesses = report.split("Strengths")
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2019.append("")
        prospect_weaknesses_2019.append("")
    else:
        if pick + 1 == 1:
            strengths_weaknesses = strengths_weaknesses[1].split("Weaknesses")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2019.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2019.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2019 = BeautifulSoup(requests.get(big_board_url + "2019").text, "html.parser")
for prospect_info in tqdm(big_board_2019.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2019:
        prospect_pick_2019.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2019.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2019.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2019.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2019.append(birthday)
        else:
            prospect_birthday_2019.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2019.append("")
            prospect_weaknesses_2019.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2019.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2019.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:46<00:00,  2.15it/s]


In [64]:
draft_2019_df = pd.DataFrame({"Name": prospect_name_2019, "Pick": prospect_pick_2019,
                              "Strengths": prospect_strengths_2019, "Weaknesses": prospect_weaknesses_2019,
                              "Height": prospect_height_2019, "Weight": prospect_weight_2019, "Birthday": prospect_birthday_2019})

In [65]:
draft_2019_df["Height"] = draft_2019_df["Height"].apply(height_to_inches)
draft_2019_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Zion Williamson,1,Incredibly unique prospect from a physical att...,Playing style and weight puts a lot of pressur...,78,280,07/06/00
1,Ja Morant,2,"Dynamic athlete and playmaker, adept at creati...",Developing the ability to consistently knock d...,75,175,08/10/99
2,RJ Barrett,3,Lefty 2-guard who plays the game with a lot of...,May have peaked to some degree in high school ...,79,210,06/14/00
3,DeAndre Hunter,4,Athletic forward with a versatile skill set … ...,There is not one clear thing that he does elit...,79,225,12/01/97
4,Darius Garland,5,6’3 point guard … Great speed and quickness … ...,Has decent but not great length with a 6’5 win...,74,175,01/26/00
...,...,...,...,...,...,...,...
97,Dedric Lawson,-1,,,80,235,10/01/97
98,Bennie Boatwright,-1,,,82,230,07/13/96
99,CJ Massinburg,-1,,,75,195,04/14/97
100,Jarrey Foster,-1,,,78,220,12/09/96


In [66]:
draft_2019_date = datetime(year = 2019, month = 6, day = 20)
draft_2019_df["Age"] = draft_2019_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2019_date))
draft_2019_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Zion Williamson,1,Incredibly unique prospect from a physical att...,Playing style and weight puts a lot of pressur...,78,280,07/06/00,18.97
1,Ja Morant,2,"Dynamic athlete and playmaker, adept at creati...",Developing the ability to consistently knock d...,75,175,08/10/99,19.87
2,RJ Barrett,3,Lefty 2-guard who plays the game with a lot of...,May have peaked to some degree in high school ...,79,210,06/14/00,19.03
3,DeAndre Hunter,4,Athletic forward with a versatile skill set … ...,There is not one clear thing that he does elit...,79,225,12/01/97,21.56
4,Darius Garland,5,6’3 point guard … Great speed and quickness … ...,Has decent but not great length with a 6’5 win...,74,175,01/26/00,19.41
...,...,...,...,...,...,...,...,...
97,Dedric Lawson,-1,,,80,235,10/01/97,21.73
98,Bennie Boatwright,-1,,,82,230,07/13/96,22.95
99,CJ Massinburg,-1,,,75,195,04/14/97,22.20
100,Jarrey Foster,-1,,,78,220,12/09/96,22.54


In [67]:
draft_2019_df.drop(columns = "Birthday").to_csv("data/2019_nba_draft_prospects.csv", index = None)

# 2020

In [68]:
prospect_name_2020 = []
prospect_strengths_2020 = []
prospect_weaknesses_2020 = []
prospect_pick_2020 = []
prospect_height_2020 = []
prospect_weight_2020 = []
prospect_birthday_2020 = []
prospect_link_2020 = set()
actual_draft_2020 = BeautifulSoup(requests.get(actual_draft_url + "2020").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2020.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2020.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2020.add(link)
    prospect_name_2020.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2020.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2020.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2020.append(birthday)
    else:
        prospect_birthday_2020.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2020.append("")
        prospect_weaknesses_2020.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2020.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2020.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2020 = BeautifulSoup(requests.get(big_board_url + "2020").text, "html.parser")
for prospect_info in tqdm(big_board_2020.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2020:
        prospect_pick_2020.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2020.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2020.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2020.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2020.append(birthday)
        else:
            prospect_birthday_2020.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2020.append("")
            prospect_weaknesses_2020.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2020.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2020.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


In [69]:
prospect_birthday_2020[50] = "05/23/98"
prospect_birthday_2020[51] = "01/06/01"
prospect_birthday_2020[59] = "05/15/96"
prospect_birthday_2020[65] = "07/21/98"
prospect_birthday_2020[100] = "09/20/97"

In [70]:
draft_2020_df = pd.DataFrame({"Name": prospect_name_2020, "Pick": prospect_pick_2020,
                              "Strengths": prospect_strengths_2020, "Weaknesses": prospect_weaknesses_2020,
                              "Height": prospect_height_2020, "Weight": prospect_weight_2020, "Birthday": prospect_birthday_2020})

In [71]:
draft_2020_df["Height"] = draft_2020_df["Height"].apply(height_to_inches)
draft_2020_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Anthony Edwards,1,"A rangy, 6’5 225 guard prospect with a truly f...",Edwards has to improve his assertiveness on bo...,77,225,08/05/01
1,James Wiseman,2,Wiseman has physical tools that really stand o...,The obvious elephant in the room is that he go...,85,235,03/31/01
2,LaMelo Ball,3,A physically gifted lead guard prospect with p...,His overall jump shot lacks any real consisten...,78,180,08/22/01
3,Patrick Williams,4,"An explosive, “freakish” combo forward with in...",There are question marks about his want to as ...,79,215,08/26/01
4,Isaac Okoro,5,Fits the prototype physically of an NBA small ...,Okoro is a work in progress when it comes to h...,78,215,01/26/01
...,...,...,...,...,...,...,...
98,Freddie Gillespie,-1,,,81,245,06/14/97
99,Trevelin Queen,-1,,,78,190,02/25/97
100,Nathan Knight,-1,,,81,235,09/20/97
101,Anthony Lamb,-1,,,78,225,01/20/98


In [72]:
draft_2020_date = datetime(year = 2020, month = 11, day = 18)
draft_2020_df["Age"] = draft_2020_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2020_date))
draft_2020_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Anthony Edwards,1,"A rangy, 6’5 225 guard prospect with a truly f...",Edwards has to improve his assertiveness on bo...,77,225,08/05/01,19.30
1,James Wiseman,2,Wiseman has physical tools that really stand o...,The obvious elephant in the room is that he go...,85,235,03/31/01,19.65
2,LaMelo Ball,3,A physically gifted lead guard prospect with p...,His overall jump shot lacks any real consisten...,78,180,08/22/01,19.25
3,Patrick Williams,4,"An explosive, “freakish” combo forward with in...",There are question marks about his want to as ...,79,215,08/26/01,19.24
4,Isaac Okoro,5,Fits the prototype physically of an NBA small ...,Okoro is a work in progress when it comes to h...,78,215,01/26/01,19.82
...,...,...,...,...,...,...,...,...
98,Freddie Gillespie,-1,,,81,245,06/14/97,23.45
99,Trevelin Queen,-1,,,78,190,02/25/97,23.75
100,Nathan Knight,-1,,,81,235,09/20/97,23.18
101,Anthony Lamb,-1,,,78,225,01/20/98,22.84


In [73]:
draft_2020_df.drop(columns = "Birthday").to_csv("data/2020_nba_draft_prospects.csv", index = None)

# 2021

In [74]:
prospect_name_2021 = []
prospect_strengths_2021 = []
prospect_weaknesses_2021 = []
prospect_pick_2021 = []
prospect_height_2021 = []
prospect_weight_2021 = []
prospect_birthday_2021 = []
prospect_link_2021 = set()
actual_draft_2021 = BeautifulSoup(requests.get(actual_draft_url + "2021").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2021.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2021.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2021.add(link)
    prospect_name_2021.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2021.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2021.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2021.append(birthday)
    else:
        prospect_birthday_2021.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2021.append("")
        prospect_weaknesses_2021.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2021.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2021.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2021 = BeautifulSoup(requests.get(big_board_url + "2021").text, "html.parser")
for prospect_info in tqdm(big_board_2021.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2021:
        prospect_pick_2021.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2021.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2021.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2021.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2021.append(birthday)
        else:
            prospect_birthday_2021.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2021.append("")
            prospect_weaknesses_2021.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2021.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2021.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


In [75]:
prospect_birthday_2021[77] = "09/17/97"

In [76]:
draft_2021_df = pd.DataFrame({"Name": prospect_name_2021, "Pick": prospect_pick_2021,
                              "Strengths": prospect_strengths_2021, "Weaknesses": prospect_weaknesses_2021,
                              "Height": prospect_height_2021, "Weight": prospect_weight_2021, "Birthday": prospect_birthday_2021})

In [77]:
draft_2021_df["Height"] = draft_2021_df["Height"].apply(height_to_inches)
draft_2021_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Cade Cunningham,1,A prized 6’8 220+ lb lead guard prospect who h...,"While he is a physically gifted prospect, Cunn...",80,220,09/25/01
1,Jalen Green,2,Green is a high upside prospect with elite ath...,"Green is a good shooter, but is not yet a snip...",77,180,02/09/02
2,Evan Mobley,3,Mobley has a chance to be a generational bigma...,The biggest hurdle for Evan Mobley is himself....,84,215,06/18/01
3,Scottie Barnes,4,Versatility and power are the two greatest str...,The biggest thing holding Barnes back from a h...,80,225,07/31/01
4,Jalen Suggs,5,"Lead guard with great leadership, toughness an...",The most glaring weakness for the star freshma...,76,205,06/03/01
...,...,...,...,...,...,...,...
97,Oscar da Silva,-1,,,82,225,09/21/98
98,Javonte Smart,-1,6’4.5 combo guard … Good length for position w...,Solid athlete but isn’t incredibly explosive …...,76,190,06/03/99
99,DJ Carton,-1,,,74,200,08/05/00
100,Denzel Mahoney,-1,,,77,220,01/18/98


In [78]:
draft_2021_date = datetime(year = 2021, month = 7, day = 29)
draft_2021_df["Age"] = draft_2021_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2021_date))
draft_2021_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Cade Cunningham,1,A prized 6’8 220+ lb lead guard prospect who h...,"While he is a physically gifted prospect, Cunn...",80,220,09/25/01,19.85
1,Jalen Green,2,Green is a high upside prospect with elite ath...,"Green is a good shooter, but is not yet a snip...",77,180,02/09/02,19.48
2,Evan Mobley,3,Mobley has a chance to be a generational bigma...,The biggest hurdle for Evan Mobley is himself....,84,215,06/18/01,20.13
3,Scottie Barnes,4,Versatility and power are the two greatest str...,The biggest thing holding Barnes back from a h...,80,225,07/31/01,20.01
4,Jalen Suggs,5,"Lead guard with great leadership, toughness an...",The most glaring weakness for the star freshma...,76,205,06/03/01,20.17
...,...,...,...,...,...,...,...,...
97,Oscar da Silva,-1,,,82,225,09/21/98,22.87
98,Javonte Smart,-1,6’4.5 combo guard … Good length for position w...,Solid athlete but isn’t incredibly explosive …...,76,190,06/03/99,22.17
99,DJ Carton,-1,,,74,200,08/05/00,20.99
100,Denzel Mahoney,-1,,,77,220,01/18/98,23.54


In [79]:
draft_2021_df.drop(columns = "Birthday").to_csv("data/2021_nba_draft_prospects.csv", index = None)

# 2022

In [80]:
prospect_name_2022 = []
prospect_strengths_2022 = []
prospect_weaknesses_2022 = []
prospect_pick_2022 = []
prospect_height_2022 = []
prospect_weight_2022 = []
prospect_birthday_2022 = []
prospect_link_2022 = set()
actual_draft_2022 = BeautifulSoup(requests.get(actual_draft_url + "2022").text, "html.parser")
for pick, prospect_info in enumerate(tqdm(actual_draft_2022.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")}))):
    prospect_pick_2022.append(pick + 1)
    link = prospect_info.attrs["href"]
    prospect_link_2022.add(link)
    prospect_name_2022.append(str(prospect_info.string))
    page = BeautifulSoup(requests.get(link).text, "html.parser")
    profile = page.find("div", attrs = {"class": "player-profile"})
    prospect_height_2022.append(profile.find("span", attrs = {"class": "player-height"}).string)
    prospect_weight_2022.append(profile.find("span", attrs = {"class": "player-weight"}).string)
    birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
    if birthday.startswith("0") or birthday.startswith("1"):
        prospect_birthday_2022.append(birthday)
    else:
        prospect_birthday_2022.append("")
    report = page.find("div", attrs = {"id": "analysis"}).text
    strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
    if len(strengths_weaknesses) == 1:
        prospect_strengths_2022.append("")
        prospect_weaknesses_2022.append("")
    else:
        strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
        prospect_strengths_2022.append(strengths_weaknesses[0].strip())
        weaknesses = strengths_weaknesses[1]
        weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
        prospect_weaknesses_2022.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())
big_board_2022 = BeautifulSoup(requests.get(big_board_url + "2022").text, "html.parser")
for prospect_info in tqdm(big_board_2022.find_all("a", attrs = {"href": re.compile("^https://www.nbadraft.net/players")})):
    link = prospect_info.attrs["href"]
    if link not in prospect_link_2022:
        prospect_pick_2022.append(-1)
        page = BeautifulSoup(requests.get(link).text, "html.parser")
        prospect_name_2022.append(page.find("h1", attrs = {"class": "player-name"}).text.split(" - ")[-1].strip())
        profile = page.find("div", attrs = {"class": "player-profile"})
        prospect_height_2022.append(profile.find("span", attrs = {"class": "player-height"}).string)
        prospect_weight_2022.append(profile.find("span", attrs = {"class": "player-weight"}).string)
        birthday = profile.find("div", attrs = {"class": "div-table-cell attribute-value"}).text.split()[0]
        if birthday.startswith("0") or birthday.startswith("1"):
            prospect_birthday_2022.append(birthday)
        else:
            prospect_birthday_2022.append("")
        report = page.find("div", attrs = {"id": "analysis"}).text
        strengths_weaknesses = re.split("Str?eng[th][th]s?:", report)
        if len(strengths_weaknesses) == 1:
            prospect_strengths_2022.append("")
            prospect_weaknesses_2022.append("")
        else:
            strengths_weaknesses = re.split("We[ae]s?kness[es]?[es]?:", strengths_weaknesses[1])
            prospect_strengths_2022.append(strengths_weaknesses[0].strip())
            weaknesses = strengths_weaknesses[1]
            weaknesses = re.sub('Notes:.+|Outlook:.+|YouTube.+|Overall:.+', "", weaknesses)
            prospect_weaknesses_2022.append(re.sub('[A-Z][a-z]+\s[A-Z][a-z]+\s(–\s)?[0-9][0-9]?/[0-9][0-9]?/[0-9][0-9]([0-9][0-9])?.*', "", weaknesses).strip())

100%|██████████| 100/100 [00:46<00:00,  2.13it/s]


In [81]:
draft_2022_df = pd.DataFrame({"Name": prospect_name_2022, "Pick": prospect_pick_2022,
                              "Strengths": prospect_strengths_2022, "Weaknesses": prospect_weaknesses_2022,
                              "Height": prospect_height_2022, "Weight": prospect_weight_2022, "Birthday": prospect_birthday_2022})

In [82]:
draft_2022_df["Height"] = draft_2022_df["Height"].apply(height_to_inches)
draft_2022_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday
0,Paolo Banchero,1,"Has good size and length for his position, sta...","For all of his offensive gifts, still has room...",82,250,11/12/02
1,Chet Holmgren,2,Extremely skilled frontcourt player who can in...,Lack of physical strength (195 lbs) remains hi...,84,195,05/01/02
2,Jabari Smith,3,Wiry 6’10 combo / stretch four with long arms ...,Rebounding numbers could stand some improvemen...,82,220,05/13/03
3,Keegan Murray,4,"A 6’8 225 frontcourt player with the frame, le...",Will have some questions about his true positi...,80,225,08/19/00
4,Jaden Ivey,5,"An aggressive explosive, 6’4 200 lb combo guar...",Ivey’s hard charging energy and play style can...,76,195,02/13/02
...,...,...,...,...,...,...,...
97,Collin Gillespie,-1,,,75,195,06/25/99
98,JeeNathan Williams,-1,,,77,205,02/12/99
99,Buddy Boeheim,-1,,,78,200,11/11/99
100,Kai Sotto,-1,,,86,230,05/11/02


In [83]:
draft_2022_date = datetime(year = 2022, month = 6, day = 23)
draft_2022_df["Age"] = draft_2022_df["Birthday"].apply(lambda birthday: age_at_draft(birthday, draft_2022_date))
draft_2022_df

,Name,Pick,Strengths,Weaknesses,Height,Weight,Birthday,Age
0,Paolo Banchero,1,"Has good size and length for his position, sta...","For all of his offensive gifts, still has room...",82,250,11/12/02,19.62
1,Chet Holmgren,2,Extremely skilled frontcourt player who can in...,Lack of physical strength (195 lbs) remains hi...,84,195,05/01/02,20.16
2,Jabari Smith,3,Wiry 6’10 combo / stretch four with long arms ...,Rebounding numbers could stand some improvemen...,82,220,05/13/03,19.13
3,Keegan Murray,4,"A 6’8 225 frontcourt player with the frame, le...",Will have some questions about his true positi...,80,225,08/19/00,21.86
4,Jaden Ivey,5,"An aggressive explosive, 6’4 200 lb combo guar...",Ivey’s hard charging energy and play style can...,76,195,02/13/02,20.37
...,...,...,...,...,...,...,...,...
97,Collin Gillespie,-1,,,75,195,06/25/99,23.01
98,JeeNathan Williams,-1,,,77,205,02/12/99,23.38
99,Buddy Boeheim,-1,,,78,200,11/11/99,22.63
100,Kai Sotto,-1,,,86,230,05/11/02,20.13


In [84]:
draft_2022_df.drop(columns = "Birthday").to_csv("data/2022_nba_draft_prospects.csv", index = None)